In [16]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV,cross_val_score, KFold


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA


from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv("DataCSVS/cleanedPlayerData41-42.csv", encoding='latin1')

df.head()



df.columns

Index(['Weight', 'IS', 'OS', 'Rng', 'Fin', 'Reb', 'IDef', 'PDef', 'IQ', 'Pass',
       'Hnd', 'Drv', 'Str', 'Spd', 'Sta', 'F-M', 'F-A', 'F%', 'IS-M', 'IS-A',
       'IS%', 'MR-M', 'MR-A', 'MR%', '3P-M', '3P-A', '3P%', 'DR-M', 'DR-A',
       'DR%', 'FTM', 'FTA', 'FT%', 'RebP', 'Ast', 'Stl', 'Blk', '2OFM', '2OFA',
       '2OF%', '3OFM', '3OFA', '3OF%', 'TO', 'PF', 'DQ', 'FD',
       'Wingspan_inches', 'Height_inches', 'Vertical_float'],
      dtype='object')

In [17]:



#df = df[df['3OFA'] >= 0.4 * (df['2OFA'] + df['3OFA'])] #For Guards
df = df[(df['DR-A'] >= 75)] #attempted ~ 2 3PA a game


columns_to_drop = ['F-M', 'F-A', 'F%', 'IS-M', 'IS-A',
       'IS%', 'MR-M', 'MR-A', 'MR%', '3P-M', '3P-A', '3P%','DR-M', 'DR-A',
        'FTM', 'FTA', 'FT%', 'RebP', 'Ast', 'Stl', 'Blk', '2OFM', '2OFA',
       '2OF%', '3OFM', '3OFA', '3OF%', 'TO', 'PF', 'DQ', 'FD']



df = df.drop(columns_to_drop,axis=1)


print(df.shape)

data = df
X = data.drop('DR%', axis=1)
y = data['DR%']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



(2393, 19)


In [18]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)



model = Ridge()
model.fit(X_train_pca, y_train)

# Retrieve the coefficients from the Ridge model
coefficients = model.coef_
#print("Coefficients:", coefficients)

# Make predictions and evaluate
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test_pca)

print("R2 Score:", r2_score(y_test, y_pred))

R2 Score: 0.07818584846685983


In [19]:

import pandas as pd

# Create a DataFrame with actual and predicted values
results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

# Optionally, you can reset the index to make it easier to read
results = results.reset_index(drop=True)

# Display the DataFrame
#print(results.head())  # Shows the first few rows of the DataFrame

# Create a DataFrame with actual features from X_test
results = pd.DataFrame(X_test)

# Add actual and predicted values to this DataFrame
results['Actual'] = y_test
results['Predicted'] = y_pred

# Reset the index to align the actual and predicted values with their corresponding features
results = results.reset_index(drop=True)

# Display the DataFrame
print(results.head())  # Shows the first few rows of the DataFrame






   Weight  IS  OS  Rng  Fin  Reb  IDef  PDef  IQ  Pass  Hnd  Drv  Str  Spd  \
0     185   8  10   12   11    3     4     7  14     5   10   11    7    6   
1     185  10   9    8   14    7    11     6   8    15   10   15    8   16   
2     180   9  15    8   12   10     6    16   8    18   17   11   14   15   
3     245  10  16    6   14   15    14     7  17    11   11   13   18   12   
4     210   6  13   15   13    7    14    10  16     9   15   13   13   13   

   Sta  Wingspan_inches  Height_inches  Vertical_float  Actual  Predicted  
0    9             83.0           76.5            27.0    0.47   0.456463  
1   17             75.0           74.0            26.0    0.52   0.486338  
2    7             77.0           72.0            28.5    0.48   0.481714  
3   15             83.0           80.5            31.0    0.40   0.477036  
4    9             83.0           78.5            31.5    0.50   0.482026  


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from pygetPlayerSkills import get_player_info




# Assuming scaler, pca, rfe, and best_model are already fitted
# and X_train, X_test, y_train, y_test are defined as in the original code

# Function to preprocess and predict new data
def preprocess_and_predict(player, scaler, pca, model):
    # Convert player data to DataFrame
    player_df = pd.DataFrame([player])
    
    # Ensure the DataFrame has the same columns as the training data
    expected_columns = X_train.columns  # Columns from the training data
    
    # Check if player_df contains all required columns
    missing_columns = set(expected_columns) - set(player_df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns in player data: {missing_columns}")

    # Ensure the DataFrame columns are in the same order as expected
    player_df = player_df[expected_columns]

    # Apply the same scaling
    player_scaled = scaler.transform(player_df)

    # Apply PCA
    player_pca = pca.transform(player_scaled)

    # Apply feature selection
    #player_rfe = rfe.transform(player_pca)

    # Predict using the best model
    prediction = model.predict(player_pca)
    
    return prediction

# Get player information
player = get_player_info("http://onlinecollegebasketball.org/player/196605")

# Predict using the preprocessed player data
new_data_pred = preprocess_and_predict(player, scaler, pca, model)

# Print the predictions
print(f"Predicted value: {format(new_data_pred[0], '.2f')}")


Predicted value: 0.44
